In [1]:
%load_ext autoreload
%autoreload 2

import duckdb
import json
import os
import pandas as pd
import pathlib

from ibge import *
from utils import *

In [3]:
# Initialize DuckDB connection
conn = duckdb.connect("piper.db")
local_file = "gbif"
!ls gbif/

Piper.json  Piper.parquet


## GBIF
### Adiciono os registros no duckdb (via parquet)

In [9]:
conn.execute("DROP TABLE IF EXISTS gbif")
conn.execute(f"""
        CREATE TABLE gbif AS
        SELECT * FROM read_parquet('{local_file}', union_by_name=True)
    """)

result = conn.execute("SELECT COUNT(*) FROM gbif").fetchone()
print(f"Registros carregados na tabela: {result[0]:,}")

Registros carregados na tabela: 20


In [10]:
conn.execute("DESCRIBE gbif").df()

,column_name,column_type,null,key,default,extra
0,key,BIGINT,YES,None,None,None
1,datasetKey,VARCHAR,YES,None,None,None
2,publishingOrgKey,VARCHAR,YES,None,None,None
3,installationKey,VARCHAR,YES,None,None,None
4,hostingOrganizationKey,VARCHAR,YES,None,None,None
...,...,...,...,...,...,...
148,rightsHolder,VARCHAR,YES,None,None,None
149,habitat,VARCHAR,YES,None,None,None
150,eventTime,VARCHAR,YES,None,None,None
151,identificationVerificationStatus,VARCHAR,YES,None,None,None


In [13]:
conn.execute("SELECT * FROM gbif LIMIT 5").df()

,key,datasetKey,publishingOrgKey,installationKey,hostingOrganizationKey,publishingCountry,protocol,lastCrawled,lastParsed,crawlId,...,individualCount,lifeStage,stateProvince,modified,samplingProtocol,rightsHolder,habitat,eventTime,identificationVerificationStatus,eventType
0,1122662226,75956ee6-1a2b-4fa3-b3e8-ccda64ce6c2d,1928bdf0-f5d2-11dc-8c12-b8a03c50a862,791e3715-7452-4dd0-a906-db21a3852427,5d5e5d82-076c-4e9a-b5b0-c88d1b691a6a,FR,EML,2025-11-04T05:25:09.232+00:00,2025-11-04T14:50:39.589+00:00,195,...,NaN,None,None,None,None,None,None,None,None,None
1,1122662255,75956ee6-1a2b-4fa3-b3e8-ccda64ce6c2d,1928bdf0-f5d2-11dc-8c12-b8a03c50a862,791e3715-7452-4dd0-a906-db21a3852427,5d5e5d82-076c-4e9a-b5b0-c88d1b691a6a,FR,EML,2025-11-04T05:25:09.232+00:00,2025-11-04T14:50:41.309+00:00,195,...,NaN,None,None,None,None,None,None,None,None,None
2,1122666550,75956ee6-1a2b-4fa3-b3e8-ccda64ce6c2d,1928bdf0-f5d2-11dc-8c12-b8a03c50a862,791e3715-7452-4dd0-a906-db21a3852427,5d5e5d82-076c-4e9a-b5b0-c88d1b691a6a,FR,EML,2025-11-04T05:25:09.232+00:00,2025-11-04T14:50:39.884+00:00,195,...,NaN,None,None,None,None,None,None,None,None,None
3,1122676409,75956ee6-1a2b-4fa3-b3e8-ccda64ce6c2d,1928bdf0-f5d2-11dc-8c12-b8a03c50a862,791e3715-7452-4dd0-a906-db21a3852427,5d5e5d82-076c-4e9a-b5b0-c88d1b691a6a,FR,EML,2025-11-04T05:25:09.232+00:00,2025-11-04T14:50:49.814+00:00,195,...,NaN,None,None,None,None,None,None,None,None,None
4,1122674214,75956ee6-1a2b-4fa3-b3e8-ccda64ce6c2d,1928bdf0-f5d2-11dc-8c12-b8a03c50a862,791e3715-7452-4dd0-a906-db21a3852427,5d5e5d82-076c-4e9a-b5b0-c88d1b691a6a,FR,EML,2025-11-04T05:25:09.232+00:00,2025-11-04T14:50:41.078+00:00,195,...,NaN,None,None,None,None,None,None,None,None,None
